In [28]:
from dotenv import load_dotenv
import os

load_dotenv()
project_name = "wanted_2nd_agent_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [29]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_tavily import TavilySearch

In [30]:
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 툴만 사용해서 사실을 확인하고 한국어로 간단하게 답한다"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])
tavily_search = TavilySearch()
agent = create_openai_tools_agent(
    llm = llm,
    tools = [tavily_search],
    prompt = prompt
)

executor = AgentExecutor(
    agent=agent,
    tools = [tavily_search],
    verbose=True
)

In [31]:
result = executor.invoke({"input":"서울의 오늘 주요 뉴스 3개 요약해줘"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '서울 오늘 주요 뉴스', 'time_range': 'day', 'topic': 'news'}`


{'query': '서울 오늘 주요 뉴스', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.tradingview.com/news/reuters.com,2025:newsml_L2N3VX03J:0-news-overload-splits-markets/', 'title': 'News overload splits markets - TradingView', 'score': 0.24706931, 'published_date': 'Thu, 16 Oct 2025 04:30:00 GMT', 'content': '# News overload splits markets A look at the day ahead in European and global markets from Kevin Buckland Stock markets are rallying around Asia on Thursday, with bourses from Taipei to Seoul and Sydney climbing to fresh all-time peaks. The latest bullish signal came from Dutch semiconductor equipment maker ASML ASML, with third-quarter bookings beating market forecasts on Wednesday. President Donald Trump declared they are "in a trade war" late on Wednesday, even as his Treasury and trade chiefs offered some h

In [32]:
result = executor.invoke({"input":"석촌역에 맛집 5개 찾아줘"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '석촌역 맛집', 'search_depth': 'basic'}`


{'query': '석촌역 맛집', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.tripadvisor.co.kr/RestaurantsNear-g2024873-d9579729-Seokchon-Seongnam_Gyeonggi_do.html', 'title': '성남 석촌 역 근처 맛집 Best 10 - Tripadvisor - 트립어드바이저', 'content': '석촌 역 근처 맛집 · 1. 미오. 5.0. (74건의 리뷰). 일본식 퓨전$$$$ · 2. 바이킹스워프 롯데월드몰점. 4.5. (219건의 리뷰) · 3. 라티튜드32. 5.0. (91건의 리뷰) · 4.', 'score': 0.87706697, 'raw_content': None}, {'url': 'https://www.diningcode.com/list.dc?query=%EC%84%9D%EC%B4%8C%EC%97%AD%20%EA%B0%80%EC%84%B1%EB%B9%84%EC%A2%8B%EC%9D%80', 'title': '석촌역 가성비좋은 맛집 Top100 - 다이닝코드', 'content': '석촌역 가성비좋은맛집 (111곳) · 1. 단디 송리단길 · 2. 이랑칼국수 석촌역 · 3. 우프 잠실 · 4. 완도산회 포장마차 송파직영점 · 5. 옛촌매운갈비찜 석촌역 · 6. 수작', 'score': 0.8585411, 'raw_content': None}, {'url': 'https://www.diningcode.com/list.dc?query=%EC%84%9D%EC%B4%8C%EC%97%AD', 'title': '석촌역 맛집 Top100 - 다이닝코드', 'cont

In [33]:
from langchain_community.agent_toolkits import FileManagementToolkit
root_dir = "../10_agent_result"
file_tools = FileManagementToolkit(root_dir=root_dir).get_tools()
file_tools

[CopyFileTool(root_dir='../10_agent_result'),
 DeleteFileTool(root_dir='../10_agent_result'),
 FileSearchTool(root_dir='../10_agent_result'),
 MoveFileTool(root_dir='../10_agent_result'),
 ReadFileTool(root_dir='../10_agent_result'),
 WriteFileTool(root_dir='../10_agent_result'),
 ListDirectoryTool(root_dir='../10_agent_result')]

In [34]:
tavily_search = TavilySearch(max_results=5)

In [35]:
total_tool = file_tools + [tavily_search]
total_tool

[CopyFileTool(root_dir='../10_agent_result'),
 DeleteFileTool(root_dir='../10_agent_result'),
 FileSearchTool(root_dir='../10_agent_result'),
 MoveFileTool(root_dir='../10_agent_result'),
 ReadFileTool(root_dir='../10_agent_result'),
 WriteFileTool(root_dir='../10_agent_result'),
 ListDirectoryTool(root_dir='../10_agent_result'),
 TavilySearch(max_results=5, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'), api_base_url=None))]

In [36]:
# 시스템 프롬프트 설정하기
SYSTEM_PROMPT = """
너는 '파일 시스템 툴'과 '웹검색 툴'만을 사용하는 어시스턴트야

[규칙]
1. 최신 뉴스/링크는 Tavily로 검색한다
2. 사용자가 csv 파일로 저장을 요청하면, 다음 필드 헤더로 csv 를 만든다
    date, query, title, url, source, snippet
3. 파일명은 {YYYYMMDD}.csv 로 저장한다
4. 같은 파일명이 있다면, 덮어 쓰지말고 기존 내용을 읽어서 헤더를 유지한채 이어 붙여서 저장한다
5. 저장한 뒤에 내용을 확인해서 가장 아래 5줄 정도 정보를 출력한다

모든 답변은 한국어로 간결하게 해라
"""

In [37]:
from datetime import datetime

def get_today():
    return datetime.now().strftime("%Y%m%d")

In [38]:
get_today()

'20251017'

In [39]:
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0
)

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human","{input}"),
    ("placeholder","{agent_scratchpad}")
])

agent = create_openai_tools_agent(
    llm = llm,
    tools =total_tool,
    prompt = prompt
)

executor = AgentExecutor(
    agent=agent,
    tools = total_tool,
    verbose=True
)

In [40]:
result = executor.invoke(
    {"input": "오늘 AI 관련 뉴스 최신 5개만 요약해줘", "YYYYMMDD": get_today()}
)
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'AI', 'time_range': 'day', 'topic': 'general', 'search_depth': 'basic'}`


{'query': 'AI', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.wired.com/story/the-ai-industrys-scaling-obsession-is-headed-for-a-cliff/', 'title': "The AI Industry's Scaling Obsession Is Headed for a Cliff | WIRED", 'content': 'A new study from MIT suggests the biggest and most computationally intensive AI models may soon offer diminishing returns compared to smaller models. Leaps in efficiency, like those seen with DeepSeek’s remarkably low-cost model in January, have already served as a reality check for the AI industry, which is accustomed to burning massive amounts of compute. While the MIT team’s prediction might not hold if, for example, new training methods like reinforcement learning produce surprising new results, they suggest that big AI firms will have less of an edge in the

In [41]:
result = executor.invoke({
    "input" : "오늘 AI 관련 뉴스 최신 5개만 요약해주고 그내용을 csv 로 저장해줘",
    "YYYYMMDD" : get_today()
})
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'AI 최신 뉴스', 'search_depth': 'advanced', 'time_range': 'day', 'topic': 'news'}`


{'query': 'AI 최신 뉴스', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://analyticsindiamag.com/ai-news-updates/matters-ai-raises-%E2%82%B955-cr-to-build-self-learning-ai-security-engineer/', 'title': 'Matters.AI Raises ₹55 Cr to Build Self-Learning ‘AI Security Engineer’ - Analytics India Magazine', 'score': 0.64719695, 'published_date': 'Thu, 16 Oct 2025 05:48:48 GMT', 'content': 'About us\n Indian IT\n Startups\n AI News\n\nSign In\n\n \n\n Published on October 16, 2025\n In   AI News\n\n# Matters.AI Raises ₹55 Cr to Build Self-Learning ‘AI Security Engineer’\n\n“The world doesn’t need another dashboard screaming alerts; it needs a system that can think,” Keshava Murthy said.\n\n By Merin Susan John [...] Microsoft Takes on Google’s Nano Banana and OpenAI’s GPT-Image-1 with New AI Model

KeyboardInterrupt: 

In [ ]:
result = executor.invoke({
    "input" : "오늘 패션 관련 뉴스 최신 5개만 요약해주고 그 내용을 csv 로 저장해줘. 그리고 하위 5개 보여줘",
    "YYYYMMDD" : get_today()
})
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '패션 최신 뉴스', 'topic': 'news', 'search_depth': 'advanced', 'time_range': 'day', 'include_images': False}`


{'query': '패션 최신 뉴스', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.businessoffashion.com/articles/global-markets/budapest-central-european-fashion-week-talent-incubator-nanushka-vanguards-group/', 'title': 'How Budapest Central European Fashion Week Became a Talent Incubator - The Business of Fashion', 'score': 0.791153073310852, 'published_date': 'Thu, 16 Oct 2025 04:00:00 GMT', 'content': 'Budapest Central European Fashion Week is positioning the Hungarian capital as a gateway to Central Europe’s fashion market — giving regional design talent a platform to showcase their unique collections to international buyers and media. * Fashion Show This week’s round-up of global markets fashion business news also features China’s Shan Future Forum and Saudi Arab

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///../10_agent_result/wanted3.db")
toolkit = SQLDatabaseToolkit(db=db,llm=llm).get_tools()
toolkit

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x151149450>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x151149450>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x151149450>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [47]:
SYSTEM_PROMPT = """
1. 사용 가능한 도구 (Available Tools)
당신은 다음 두 가지 유형의 도구를 사용할 수 있습니다.

웹 검색 도구 (Tavily Search):

주어진 검색어로 웹을 검색하고, 관련성 높은 결과를 JSON 형식으로 반환합니다. 이 결과를 데이터베이스에 저장할 정보의 소스로 사용해야 합니다.
SQL 데이터베이스 도구 (SQLDatabaseToolkit):
sqlite:///../10_agent_result/wanted2.db 데이터베이스와 상호작용하는 도구 세트입니다.
주요 기능:
list_tables: 데이터베이스에 있는 테이블 목록을 확인합니다.
schema_of_tables: 특정 테이블의 스키마(컬럼명, 데이터 타입 등)를 조회합니다. 데이터를 삽입하기 전, 반드시 이 기능으로 목표 테이블의 구조를 확인해야 합니다.
query_sql_db: SQL 쿼리를 실행합니다. 검색 결과를 저장하기 위해 INSERT 문을 실행하는 것이 주된 용도입니다.
query_checker_sql_db: 실행할 SQL 쿼리의 문법이 올바른지 검사합니다.

2. 핵심 작업 흐름 (Core Workflow)
당신은 다음 단계를 반드시 순서대로 따라야 합니다.
요청 분석 (Analyze Request):
사용자의 질문을 명확히 이해하여 웹에서 어떤 정보를 검색해야 하는지 핵심 키워드를 파악합니다.
웹 검색 수행 (Perform Web Search):
Tavily Search 도구를 사용하여 요청에 맞는 정보를 검색합니다.
테이블이 없을경우 적당하게 만듭니다
데이터베이스 구조 확인 (Inspect DB Schema):
list_tables 도구로 저장할 테이블이 무엇인지 확인합니다.
schema_of_tables 도구로 해당 테이블의 정확한 스키마(컬럼명, 데이터 타입)를 파악합니다. 이 단계는 데이터를 올바른 형식으로 가공하기 위해 필수적입니다.
데이터 가공 및 SQL 생성 (Process Data & Generate SQL):
웹 검색 결과(JSON)를 파싱(parsing)하여 데이터베이스 테이블 스키마에 맞게 데이터를 정리합니다.
정리된 데이터를 테이블에 삽입하기 위한 SQL INSERT 문을 생성합니다.
데이터베이스에 저장 (Save to Database):
생성한 INSERT 문을 query_sql_db 도구로 실행하여 데이터베이스에 검색 결과를 저장합니다.
최종 보고 (Final Report):
데이터 저장이 성공적으로 완료되었음을 사용자에게 명확하고 간결하게 보고합니다. (예: "요청하신 'X'에 대한 웹 검색 결과를 DB에 성공적으로 저장했습니다.")

3. 규칙 및 제약 조건 (Rules & Constraints)
추측 금지: 절대로 정보를 임의로 생성하거나 추측하지 마세요. 모든 데이터는 반드시 Tavily Search를 통해 얻은 결과에 기반해야 합니다.
도구 우선 사용: 정보 검색과 데이터 저장을 위해서는 반드시 제공된 도구를 사용해야 합니다.
작업 흐름 준수: 반드시 위에 명시된 핵심 작업 흐름의 순서를 지켜야 합니다. 특히, 스키마를 확인하기 전에 INSERT를 시도해서는 안 됩니다.
오류 처리: SQL 쿼리 실행 중 오류가 발생하면, query_checker_sql_db를 사용해 쿼리를 검사하고 수정하여 다시 시도하세요.
간결한 최종 답변: 사용자와의 최종 상호작용에서는 내부적인 생각이나 도구 사용 과정을 장황하게 설명하지 말고, 작업 완료 여부와 결과만 간결하게 전달하세요.
"""

In [48]:
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

total_tool = toolkit + [tavily_search]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_openai_tools_agent(llm=llm, tools=total_tool, prompt=prompt)

executor = AgentExecutor(agent=agent, tools=total_tool, verbose=True)

In [49]:
result = executor.invoke({"input":"최신 NPU 관련 뉴스 5개 찾아서 알려줘"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'latest NPU news', 'search_depth': 'advanced', 'topic': 'general', 'include_favicon': True}`


{'query': 'latest NPU news', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.techpowerup.com/news-tags/NPU', 'title': "News Posts matching 'NPU'", 'content': '# Intel "Nova Lake-S" Core Ultra 3, Ultra 5, Ultra 7, and Ultra 9 Core Configurations Surface\n\n# QNAP Unveils the New QAI-M100 and QAI-U100 Edge AI Accelerators\n\n# AMD and Stability AI Enable Local AI Image Generation on NPU-Powered Laptops\n\n# Intel Skips NPU Upgrade for Arrow Lake Refresh, AMD Cancels Medusa Halo in Latest Rumors\n\n# Intel Plans "Arrow Lake Refresh" for H2\u202f2025 with Higher Clocks and Upgraded NPU\n\n# AMD UDNA Graphics Architecture to Power Next-Gen Xbox and PlayStation [...] # News Posts matching #NPU\n\n# Synaptics Launches the Next Generation of Astra Multimodal GenAI Edge Proces